In [4]:
import pandas as pd
import os

# Base path
base_read_path = "../../results/pipeline results/5884paras_598qna/copies/"
lqr_path = os.path.join(base_read_path, "modLQR_processed_results - Copy.csv")

# Read data
lqr_df = pd.read_csv(lqr_path)

# Map level to true label
def get_true_label(level):
    if level.strip().lower() == "easy":
        return "singlehop"
    elif level.strip().lower() in ["medium", "hard"]:
        return "multihop"
    return None  # for safety

lqr_df["true_label"] = lqr_df["level"].apply(get_true_label)

# Strip/clean predicted classification as well
lqr_df["classification"] = lqr_df["classification"].str.strip().str.lower()

# Compute correctness
lqr_df["is_correct"] = lqr_df["classification"] == lqr_df["true_label"]

# Overall accuracy
overall_accuracy = lqr_df["is_correct"].mean()

# Accuracy by level
level_grouped = lqr_df.groupby("level")["is_correct"].agg(['count', 'sum'])
level_grouped["accuracy"] = level_grouped["sum"] / level_grouped["count"]

# Display results
print("Overall Accuracy: {:.2f}%".format(overall_accuracy * 100))
print("\nAccuracy by Level:")
print(level_grouped[["accuracy"]].rename(columns={"accuracy": "Accuracy (%)"}) * 100)


Overall Accuracy: 29.43%

Accuracy by Level:
        Accuracy (%)
level               
easy       81.632653
hard       19.200000
medium     19.200000


In [7]:
import ollama
def classify_urdu_question(urdu_question):
    prompt = f"""You are given a question written in **Urdu**. Your task is to classify it as either **SINGLEHOP** or **MULTIHOP**. 

Definitions:
- A **SINGLEHOP** question can be answered using a single fact, sentence, or document. No reasoning across steps is needed.
- A **MULTIHOP** question requires you to combine multiple facts, reason step by step, or connect information across different sources.

⚠️ Important:
- Do **not** rely on the question's length.
- Think carefully: Does answering the question require answering something first, then using that answer to proceed? If yes → it's MULTIHOP.

👇 Follow this logic:
If the question has two parts or needs cause-effect or event reasoning → MULTIHOP.  
If the question just asks for one name, date, fact, or location → SINGLEHOP.

Examples:

🔹 SINGLEHOP Example 1:
سوال: "پاکستان کے وزیر اعظم کا نام کیا ہے؟"
جواب: SINGLEHOP

🔹 SINGLEHOP Example 2:
سوال: "پانی کا کیمیائی فارمولا کیا ہے؟"
جواب: SINGLEHOP

🔸 MULTIHOP Example 1 (location + reasoning):
سوال: "نلسن منڈیلا کو کس جیل میں رکھا گیا اور وہ جیل کس جزیرے پر تھی؟"
جواب: MULTIHOP

🔸 MULTIHOP Example 2 (education + identity chaining):
سوال: "ایپل کمپنی کے شریک بانی نے کس یونیورسٹی میں تعلیم حاصل کی، اور اس یونیورسٹی کے معروف فارغ التحصیل افراد میں کون شامل ہیں؟"
جواب: MULTIHOP

🔸 MULTIHOP Example 3 (historical causality):
سوال: "1857 کی جنگ آزادی کے بعد برصغیر پر کس نے حکومت کی؟ اور کس معاہدے کے تحت؟"
جواب: MULTIHOP

🔸 MULTIHOP Example 4 (person + timeline + award):
سوال: "عبدالستار ایدھی نے کب فلاحی کام شروع کیے اور ان کو کس اہم ایوارڈ سے نوازا گیا؟"
جواب: MULTIHOP

🔸 MULTIHOP Example 5 (place + organization + year):
سوال: "پاکستان نے اقوام متحدہ میں شمولیت کب اختیار کی، اور اس وقت وزیر خارجہ کون تھا؟"
جواب: MULTIHOP

---

Now read the following Urdu question and think step by step:  
Does it require combining multiple facts or reasoning in two parts?

سوال: "{urdu_question}"
جواب:"""
    try:
        response = ollama.chat(
            model='llama3:8b',
            messages=[{"role": "user", "content": prompt}]
        )
        reply = response['message']['content'].strip().lower()

        # Normalize output
        if 'multihop' in reply:
            return 'multihop'
        elif 'singlehop' in reply or 'simple' in reply:
            return 'singlehop'

        print(f"⚠️ Unexpected response: {reply}")
        return "unknown"

    except Exception as e:
        print(f"❌ Error processing question: {urdu_question}\n↪ {e}")
        return "error"


Synthetatic Dataset

In [8]:
classification_test_data_urdu = {
    # === 20 SINGLEHOP ===
    "پاکستان کا دارالحکومت کیا ہے؟": "singlehop",
    "قرآن پاک میں کتنی سورتیں ہیں؟": "singlehop",
    "اردو کا قومی شاعر کون ہے؟": "singlehop",
    "چاند پر سب سے پہلے کون گیا؟": "singlehop",
    "علامہ اقبال کا یوم پیدائش کب ہے؟": "singlehop",
    "لاہور کس ملک میں واقع ہے؟": "singlehop",
    "پانی کا کیمیکل فارمولا کیا ہے؟": "singlehop",
    "پاکستان کے سب سے بڑے شہر کا نام کیا ہے؟": "singlehop",
    "قائداعظم کا اصلی نام کیا تھا؟": "singlehop",
    "انگریزی حروف تہجی میں کتنے حروف ہوتے ہیں؟": "singlehop",
    "قومی ترانے کے شاعر کا نام کیا ہے؟": "singlehop",
    "اردو کا پہلا افسانہ کون سا تھا؟": "singlehop",
    "سب سے بڑی انسانی آبادی والا ملک کون سا ہے؟": "singlehop",
    "پاکستان میں سب سے طویل دریا کون سا ہے؟": "singlehop",
    "اردو زبان میں حروف تہجی کتنے ہیں؟": "singlehop",
    "قرآن پاک کا پہلا حرف کیا ہے؟": "singlehop",
    "یورپ کا سب سے بڑا ملک کون سا ہے؟": "singlehop",
    "اسلام کا دوسرا رکن کون سا ہے؟": "singlehop",
    "پاکستان کب آزاد ہوا؟": "singlehop",
    "پاکستان کا قومی جانور کیا ہے؟": "singlehop",

    # === 30 MULTIHOP ===
    "محمد علی جناح نے کون سی یونیورسٹی سے تعلیم حاصل کی اور وہاں ان کا کیا شعبہ تھا؟": "multihop",
    "شاعر مشرق نے بانگ درا کب لکھی اور اس میں کون سی مشہور نظم شامل ہے؟": "multihop",
    "علامہ اقبال نے جس کانفرنس میں شرکت کی، وہ کس ملک میں منعقد ہوئی اور اس کا مقصد کیا تھا؟": "multihop",
    "پاکستان کے پہلے وزیر اعظم کا نام اور ان کی وفات کا سال کیا ہے؟": "multihop",
    "قرآن پاک کی سب سے طویل سورہ کون سی ہے اور اس میں کتنی آیات ہیں؟": "multihop",
    "جنرل ایوب خان کے دور میں کون سا اہم صنعتی منصوبہ مکمل ہوا اور اس کا مقام کیا تھا؟": "multihop",
    "اقوام متحدہ کی رکنیت پاکستان کو کب ملی اور اس وقت وزیر خارجہ کون تھا؟": "multihop",
    "اردو زبان کے سب سے پہلے شاعر کون تھے اور ان کا معروف کلام کیا تھا؟": "multihop",
    "انگریزوں کے خلاف پہلی جنگ آزادی کب شروع ہوئی اور کس شہر سے اس کا آغاز ہوا؟": "multihop",
    "پاکستان کے ان شہروں کے نام بتائیں جنہیں صنعتی شہروں کے طور پر جانا جاتا ہے اور وہاں کی بڑی صنعتیں کون سی ہیں؟": "multihop",
    "مولانا محمد علی جوہر نے کون سے اخبار کی بنیاد رکھی اور اس کا مقصد کیا تھا؟": "multihop",
    "سندھ طاس معاہدہ کب ہوا اور اس پر دستخط کرنے والے کون تھے؟": "multihop",
    "علامہ اقبال نے کب یورپ کا سفر کیا اور وہاں کن مضامین پر لیکچر دیے؟": "multihop",
    "بھارت اور پاکستان کے درمیان ہونے والی پہلی جنگ کب ہوئی اور اس کا نتیجہ کیا نکلا؟": "multihop",
    "پاکستان کے آئین میں اسلامی دفعات کب شامل کی گئیں اور کس حکومت کے دور میں؟": "multihop",
    "اقبال کے مشہور خطبات کہاں پیش کیے گئے اور ان کا اصل مقصد کیا تھا؟": "multihop",
    "اردو ادب کی پہلی ناول نگار خاتون کون تھیں اور ان کا مشہور ناول کون سا ہے؟": "multihop",
    "پاکستان میں سب سے پہلے بجلی کب پیدا کی گئی اور کس شہر میں؟": "multihop",
    "علامہ شبلی نعمانی نے کس ادارے میں تعلیم حاصل کی اور وہاں ان کا کیا کردار تھا؟": "multihop",
    "پاکستان میں پہلا ایٹمی دھماکہ کب ہوا اور اس کا مقام کیا تھا؟": "multihop",
    "پاکستان میں سب سے پہلے اولمپکس میں کون سا کھلاڑی شریک ہوا اور کس کھیل میں؟": "multihop",
    "اردو کے مشہور شاعر میر تقی میر نے کس دور میں شاعری کی اور ان کا کلام کن موضوعات پر تھا؟": "multihop",
    "پہلی مردم شماری کب ہوئی اور پاکستان کی اس وقت کی آبادی کیا تھی؟": "multihop",
    "ہجری تقویم کا آغاز کس واقعے سے ہوا اور وہ کس سال میں پیش آیا؟": "multihop",
    "پاکستان میں سب سے زیادہ شرح خواندگی کس صوبے میں ہے اور اس کی وجہ کیا ہے؟": "multihop",
    "قائداعظم نے پاکستان کے پہلے یوم آزادی پر کون سی تقریر کی اور اس میں کیا نکات بیان کیے؟": "multihop",
    "اردو زبان میں نثر کی ابتداء کب ہوئی اور اس کی پہلی تحریر کون سی تھی؟": "multihop",
    "پاکستان کا سب سے پہلا تعلیمی ادارہ کون سا تھا اور اس کی بنیاد کب رکھی گئی؟": "multihop",
    "جنرل ضیاء الحق کے دور میں کون سی اسلامی اصلاحات متعارف کرائی گئیں اور ان کے اثرات کیا تھے؟": "multihop",
    "پاکستان میں سی پیک منصوبہ کب شروع ہوا اور اس کا بنیادی مقصد کیا ہے؟": "multihop",
}


In [9]:
results = []
correct_singlehop = 0
correct_multihop = 0
total_singlehop = 0
total_multihop = 0

for idx, (question, true_label) in enumerate(classification_test_data_urdu.items(), 1):
    print(f"\n🔍 Query {idx}/{len(classification_test_data_urdu)}:")
    print(f"سوال: {question}")
    
    predicted = classify_urdu_question(question)

    results.append({
        "question": question,
        "true_label": true_label,
        "predicted_label": predicted
    })

    if true_label == "singlehop":
        total_singlehop += 1
        if predicted == "singlehop":
            correct_singlehop += 1
    elif true_label == "multihop":
        total_multihop += 1
        if predicted == "multihop":
            correct_multihop += 1

# Accuracy summary
singlehop_accuracy = correct_singlehop / total_singlehop if total_singlehop else 0
multihop_accuracy = correct_multihop / total_multihop if total_multihop else 0
overall_accuracy = (correct_singlehop + correct_multihop) / (total_singlehop + total_multihop)

print(f"\n✅ اردو سوالات کی درجہ بندی کا خلاصہ:")
print(f"- SINGLEHOP: {correct_singlehop}/{total_singlehop} ({singlehop_accuracy:.2%})")
print(f"- MULTIHOP: {correct_multihop}/{total_multihop} ({multihop_accuracy:.2%})")
print(f"- OVERALL: {(correct_singlehop + correct_multihop)}/{len(classification_test_data_urdu)} ({overall_accuracy:.2%})")

# Optional: Convert results to DataFrame
import pandas as pd
df_results_urdu = pd.DataFrame(results)



🔍 Query 1/50:
سوال: پاکستان کا دارالحکومت کیا ہے؟

🔍 Query 2/50:
سوال: قرآن پاک میں کتنی سورتیں ہیں؟

🔍 Query 3/50:
سوال: اردو کا قومی شاعر کون ہے؟

🔍 Query 4/50:
سوال: چاند پر سب سے پہلے کون گیا؟

🔍 Query 5/50:
سوال: علامہ اقبال کا یوم پیدائش کب ہے؟

🔍 Query 6/50:
سوال: لاہور کس ملک میں واقع ہے؟

🔍 Query 7/50:
سوال: پانی کا کیمیکل فارمولا کیا ہے؟

🔍 Query 8/50:
سوال: پاکستان کے سب سے بڑے شہر کا نام کیا ہے؟

🔍 Query 9/50:
سوال: قائداعظم کا اصلی نام کیا تھا؟

🔍 Query 10/50:
سوال: انگریزی حروف تہجی میں کتنے حروف ہوتے ہیں؟

🔍 Query 11/50:
سوال: قومی ترانے کے شاعر کا نام کیا ہے؟

🔍 Query 12/50:
سوال: اردو کا پہلا افسانہ کون سا تھا؟

🔍 Query 13/50:
سوال: سب سے بڑی انسانی آبادی والا ملک کون سا ہے؟

🔍 Query 14/50:
سوال: پاکستان میں سب سے طویل دریا کون سا ہے؟

🔍 Query 15/50:
سوال: اردو زبان میں حروف تہجی کتنے ہیں؟

🔍 Query 16/50:
سوال: قرآن پاک کا پہلا حرف کیا ہے؟

🔍 Query 17/50:
سوال: یورپ کا سب سے بڑا ملک کون سا ہے؟

🔍 Query 18/50:
سوال: اسلام کا دوسرا رکن کون سا ہے؟

🔍 Query 19/50:
سوال: پ